# Redoing the AF2 template dependend metrics 
The script to calculate the template dependend metrics has changed. Also, the new ipSAE metric needs to be included. Therefore redo the AF2 template metrics

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.axes._axes import Axes
from matplotlib.figure import Figure
from pathlib import Path
from sklearn.metrics import roc_curve, roc_auc_score
import re
import filecmp
import os

import pymol
from Bio.PDB import PDBParser
from Bio.PDB.Structure import Structure as BioPy_PDBStructure
from Bio.PDB.Model import Model as BioPy_PDBModel
from Bio.PDB.PDBExceptions import PDBConstructionException
parser = PDBParser(QUIET=True)

In [2]:
# Settings

# Path to the parsed AF3 output
path_AF2_structures = Path("../ressources/AF2").resolve()
path_solved_structures = Path("../ressources/solved").resolve()

# Path to AF2 output on the luck drive
path_af2_output = Path(r"L:\imb-luckgr2\projects\AlphaFold")

In [3]:
dataAF = pd.read_csv(path_AF2_structures / "AF_metrics_all_structures.tsv", sep="\t")
dataAF.drop(columns=["RMSD_domain", "num_align_atoms_domain", "align_score_domain", "num_align_resi_domain", "RMSD_backbone_peptide", "RMSD_all_atom_peptide", "known_motif_plddt", "DockQ", "iRMS", "LRMS", "Fnonnat", "label"], inplace=True)
dataAF

,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,pDockQ,iPAE,num_chainA_intf_res,num_chainB_intf_res,num_res_res_contact,num_atom_atom_contact,num_mutation_in_motif
0,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_0,0.887117,96.107999,77.495999,91.454999,0.162263,3.311542,15,5,23,208,known minimal
1,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_1,0.871984,95.793846,73.986000,89.736111,0.145001,3.395909,13,5,20,190,known minimal
2,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_2,0.760784,95.547501,57.906001,86.585239,0.116743,6.166772,16,5,27,237,known minimal
3,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_3,0.413662,94.646667,21.510000,76.362500,0.036380,16.713730,9,3,11,83,known minimal
4,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_4,0.359078,94.830001,19.753333,72.307001,0.029969,18.696838,7,3,9,108,known minimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_0,0.298701,49.827778,73.161904,62.392307,0.046600,16.147715,18,21,44,494,random minimal DDI
3176,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_1,0.246961,54.362000,76.415000,66.390909,0.033111,17.929291,10,12,19,173,random minimal DDI
3177,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_2,0.211579,52.651538,72.910908,61.937083,0.024834,18.416771,13,11,19,101,random minimal DDI
3178,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_3,0.211144,50.398334,78.798751,66.627144,0.026651,19.303787,12,16,29,402,random minimal DDI


In [ ]:
# Parsing the file names and structures for the solved structures

dataSolved = pd.DataFrame(columns=["set", "PDB_id", "ddi_pfam_id", "path", "chainA_id", "chainB_id"])

for set_folder in [Path(path_AF2_structures / "DMI"), Path(path_AF2_structures / "DDI")]:
    set_ = set_folder.name
    for prediction_file in [p for p in set_folder.iterdir() if p.is_file() and p.suffix == ".pdb"]:
        if set_ == "DMI":
            dataSolved.loc[len(dataSolved)] = {"set" : "DMI", "PDB_id": pdb_id, "path": structure_file.relative_to(path_solved_structures), "chainA_id": "A", "chainB_id": "B"}

for structure_file in [p for p in Path(path_AF2_structures / "DMI").iterdir() if p.is_file() and p.suffix == ".pdb"]:
    pdb_id = structure_file.name.split("_")[0]
    dataSolved.loc[len(dataSolved)] = {"set" : "DMI", "PDB_id": pdb_id, "path": structure_file.relative_to(path_AF2_structures), "chainA_id": "A", "chainB_id": "B"}


for benchmark_set, run in {"known_DMI": "run37", "random_DMI": "run38", "mutations_DMI": "run43", "known_DDI": "run6", "random_DDI": "run5"}.items():
    set_ = "DMI" if "DMI" in benchmark_set else "DDI"
    print(benchmark_set)
    for folder in [p for p in (path_AF2_structures / set_ / benchmark_set).iterdir() if p.is_dir()]:
        print("\t", folder.name)
        if len(models_paths := [p for p in folder.iterdir() if p.suffix == ".pdb"]) != 5:
            print("\t\t", "Missing pdb files")
        for f in models_paths:




# DMI
for structure_file in path_solved_structures:
    pdb_id = structure_file.name.split("_")[0]
    if not len(pdb_id) == 4:
        raise RuntimeError(f"Unexpected file name {structure_file.name}")
    
    dataSolved.loc[len(dataSolved)] = {"set" : "DMI", "PDB_id": pdb_id, "path": structure_file.relative_to(path_solved_structures), "chainA_id": "A", "chainB_id": "B"}

# DDI
for structure_file in DDI_solved:
    ddi_pfam_id = "_".join(structure_file.name.split("_")[0:2])
    pdb_id = structure_file.name.split("_")[2]
    chainA_id = structure_file.name.split("_")[3][0]
    chainB_id = structure_file.name.split("_")[3][1]

    dataSolved.loc[len(dataSolved)] = {"set" : "DDI", "PDB_id": pdb_id, "ddi_pfam_id": ddi_pfam_id, "path": structure_file.relative_to(path_solved_structures), "chainA_id": chainA_id, "chainB_id": chainB_id}

dataSolved

,set,PDB_id,ddi_pfam_id,path,chainA_id,chainB_id
0,DMI,1ATP,NaN,DMI_solved_structures\1ATP_min_DMI.pdb,A,B
1,DMI,1AXC,NaN,DMI_solved_structures\1AXC_min_DMI.pdb,A,B
2,DMI,1B72,NaN,DMI_solved_structures\1B72_min_DMI.pdb,A,B
3,DMI,1B8Q,NaN,DMI_solved_structures\1B8Q_min_DMI.pdb,A,B
4,DMI,1BXX,NaN,DMI_solved_structures\1BXX_min_DMI.pdb,A,B
...,...,...,...,...,...,...
181,DDI,3ZNI,PF14447_PF00179,DDI_solved_structures\PF14447_PF00179_3ZNI_AC.pdb,A,C
182,DDI,3J7Y,PF14978_PF00327,DDI_solved_structures\PF14978_PF00327_3J7Y_oZ.pdb,o,Z
183,DDI,6D6Q,PF15985_PF10175,DDI_solved_structures\PF15985_PF10175_6D6Q_GL.pdb,G,L
184,DDI,3KZ1,PF17838_PF00071,DDI_solved_structures\PF17838_PF00071_3KZ1_BE.pdb,B,E


### 1 Parsing file name

In [4]:
# Parsing benchmark_set

benchmark_set_replace_dict = {"1": "mutations_DMI", "2" : "mutations_DMI", "approved minimal DDI": "known_DDI", "known minimal": "known_DMI", "random minimal": "random_DMI", "random minimal DDI": "random_DDI"}

dataAF["benchmark_set"] = None
dataAF["num_mutations"] = None

for i, row in dataAF.iterrows():
    if row["num_mutation_in_motif"] == "1":
        dataAF.at[i, "num_mutations"] = 1
    elif row["num_mutation_in_motif"] == "2":
        dataAF.at[i, "num_mutations"] = 2
    benchmark_set = benchmark_set_replace_dict[row["num_mutation_in_motif"]]
    dataAF.at[i, "benchmark_set"] = benchmark_set

dataAF.drop(columns=["num_mutation_in_motif"], inplace=True)

In [5]:
# Regex checks on filename
regex_paired_DMI = r"^([\w\-]+)_(\w{4})$"
regex_random_DMI = r"^M([\w\-]+)_(\w{4})\.D([\w\-]+)_(\w{4})$"
regex_mutated_DMI = r"^([\w\-]+)_(\w{4})_(\w+)\.([A-Za-z]+)$"
regex_known_extension_DMI = r"^([\w-]+)_((Mmin)|(MFL)|(M[\d]+_M[\d]+))_((DFL)|(Dmin)|(D[\d]+_D[\d]+))$"
regex_ddi_known = r"^([^\W_]+_[^\W_]+)_(\w{4})_(\w+)_resi(\d+)_resi(\d+).(\w+)_resi(\d+)_resi(\d+)$"
regex_ddi_random = r"^D1([^\W_]+_[^\W_]+)_(\w{4}).D2([^\W_]+_[^\W_]+)_(\w{4})$"


dataAF["PDB_id"] = None
dataAF["ELM_instance"] = None
dataAF["ddi_pfam_id"] = None
dataAF["PDB_id_random_paired"] = None
dataAF["ELM_instance_random_paired"] = None
dataAF["ddi_pfam_id_random_paired"] = None
dataAF["sequence_initial"] = None
dataAF["sequence_mutated"] = None
# known_extensions have not been run. Therefore exclude them here but keep the code for them
#dataAF["known_extension_motif"] = None 
#dataAF["known_extension_domain"] = None
dataAF["chainA_id"] = None
dataAF["chainB_id"] = None
dataAF["chainA_start"] = None
dataAF["chainA_end"] = None
dataAF["chainB_start"] = None
dataAF["chainB_end"] = None

for i, row in dataAF.iterrows():
    pdb_id, pdb_id_2, elm_instance, elm_instance_2, sequence, sequence_f = None, None, None, None, None, None
    known_extensionM, known_extensionD, chain1_letter, chain2_letter, ddi_pfam_id, ddi_pfam_id_random_paired = None, None, None, None, None, None
    c1_start, c1_end, c2_start, c2_end = None, None, None, None
    if (benchmark_set := row["benchmark_set"]) == "known_DMI":
        if (r1 := re.search(regex_paired_DMI, row["prediction_name"])) is not None and len(r1.groups()) == 2:
            elm_instance = r1.groups()[0]
            pdb_id = r1.groups()[1]
            chain1_letter, chain2_letter = "A", "B"
    elif benchmark_set == "random_DMI":
        if (r := re.search(regex_random_DMI, row["prediction_name"])) is not None and len(r.groups()) == 4:
            # Contraintuitive, but here before dot is motif and after dot is domain
            elm_instance_2 = r.groups()[0]
            pdb_id_2 = r.groups()[1]
            elm_instance = r.groups()[2]
            pdb_id = r.groups()[3]
            chain1_letter, chain2_letter = "A", "B"
    elif benchmark_set == "mutations_DMI":
        if (r := re.search(regex_mutated_DMI, row["prediction_name"])) is not None and len(r.groups()) == 4:
            elm_instance = r.groups()[0]
            pdb_id = r.groups()[1]
            sequence = r.groups()[2]
            sequence_f = r.groups()[3]
            chain1_letter, chain2_letter = "A", "B"
    #elif benchmark_set == "known_extension":
    #    if (r := re.search(regex_known_extension_DMI, row["prediction_name"])) is not None and len(r.groups()) == 9:
    #        elm_instance = r.groups()[0]
    #        known_extensionM = r.groups()[1]
    #        known_extensionD = r.groups()[5]
    elif benchmark_set == "known_DDI":
        if (r := re.search(regex_ddi_known, row["prediction_name"])) is not None and len(r.groups()) == 8:
            ddi_pfam_id = r.groups()[0]
            pdb_id = r.groups()[1]
            chain1_letter = r.groups()[2]
            c1_start = r.groups()[3]
            c1_end = r.groups()[4]
            chain2_letter = r.groups()[5]
            c2_start = r.groups()[6]
            c2_end = r.groups()[7]

            if pdb_id != pdb_id.upper():
                pdb_id = pdb_id.upper()
                new_prediction_name = row["prediction_name"][:r.span(2)[0]] + pdb_id + row["prediction_name"][r.span(2)[1]:]
                print(f"Fixed prediction_name in set {benchmark_set} from {row['prediction_name']} to {new_prediction_name}")
                dataAF.at[i, "prediction_name"] = new_prediction_name
    elif benchmark_set == "random_DDI":
        if (r := re.search(regex_ddi_random, row["prediction_name"])) is not None and len(r.groups()) == 4:
            ddi_pfam_id = r.groups()[0]
            pdb_id = r.groups()[1]   
            ddi_pfam_id_random_paired = r.groups()[2]
            pdb_id_2 = r.groups()[3]  
    else:
        raise RuntimeError(f"Regex failed on {row['pdb_id']}")
    
    dataAF.at[i, "PDB_id"] =  pdb_id
    dataAF.at[i, "PDB_id_random_paired"] =  pdb_id_2
    dataAF.at[i, "ELM_instance"] =  elm_instance
    dataAF.at[i, "ELM_instance_random_paired"] =  elm_instance_2
    dataAF.at[i, "sequence_initial"] =  sequence
    dataAF.at[i, "sequence_mutated"] =  sequence_f
    #dataAF.at[i, "known_extension_motif"] =  known_extensionM
    #dataAF.at[i, "known_extension_domain"] =  known_extensionD
    dataAF.at[i, "chainA_id"] =  chain1_letter
    dataAF.at[i, "chainB_id"] =  chain2_letter
    dataAF.at[i, "ddi_pfam_id"] =  ddi_pfam_id
    dataAF.at[i, "ddi_pfam_id_random_paired"] =  ddi_pfam_id_random_paired
    dataAF.at[i, "chainA_start"] =  c1_start
    dataAF.at[i, "chainA_end"] =  c1_end
    dataAF.at[i, "chainB_start"] =  c2_start
    dataAF.at[i, "chainB_end"] =  c2_end

# The chain ids as well as start and end residues for the random_ddi can be obtained from the random DDI
for i, row in dataAF[dataAF["benchmark_set"] == "random_DDI"].iterrows():
    prediction_name = row["prediction_name"]
    pdb_id, pdb_id_2 = row["PDB_id"], row["PDB_id_random_paired"]
    ddi_pfam_id, ddi_pfam_id_2 = row["ddi_pfam_id"], row["ddi_pfam_id_random_paired"]
    
    if len(list((_row1 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DDI", np.logical_and(dataAF["PDB_id"] == pdb_id, dataAF["ddi_pfam_id"] == ddi_pfam_id))])["chainA_id"])) == 0:
        print(f"Can't find {pdb_id} from {prediction_name} (random_ddi, chain A) in the known_DDI set")
        continue
    if len(list((_row2 := dataAF[np.logical_and(dataAF["benchmark_set"] == "known_DDI", np.logical_and(dataAF["PDB_id"] == pdb_id_2, dataAF["ddi_pfam_id"] == ddi_pfam_id_2))])["chainB_id"])) == 0:
        print(f"Can't find {pdb_id_2} from {prediction_name} (random_ddi, chain B) in the known_DDI set")
        continue
    dataAF.at[i, "chainA_id"] = list(_row1["chainA_id"])[0]
    dataAF.at[i, "chainA_start"] = list(_row1["chainA_start"])[0]
    dataAF.at[i, "chainA_end"] = list(_row1["chainA_end"])[0]
    dataAF.at[i, "chainB_id"] =  list(_row2["chainB_id"])[0]
    dataAF.at[i, "chainB_start"] = list(_row2["chainB_start"])[0]
    dataAF.at[i, "chainB_end"] = list(_row2["chainB_end"])[0]
print("\n", f"Rows, where the regex failed")
display(dataAF[dataAF["PDB_id"].isna()])
display(dataAF)

Fixed prediction_name in set known_DDI from PF00009_PF01873_2d74_A_resi12_resi200.B_resi21_resi133 to PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133
Fixed prediction_name in set known_DDI from PF00009_PF01873_2d74_A_resi12_resi200.B_resi21_resi133 to PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133
Fixed prediction_name in set known_DDI from PF00009_PF01873_2d74_A_resi12_resi200.B_resi21_resi133 to PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133
Fixed prediction_name in set known_DDI from PF00009_PF01873_2d74_A_resi12_resi200.B_resi21_resi133 to PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133
Fixed prediction_name in set known_DDI from PF00009_PF01873_2d74_A_resi12_resi200.B_resi21_resi133 to PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133
Fixed prediction_name in set known_DDI from PF00026_PF06394_1f34_A_resi13_resi326.B_resi62_resi120 to PF00026_PF06394_1F34_A_resi13_resi326.B_resi62_resi120
Fixed prediction_name in set known_DDI from PF00026_PF0639

,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,ELM_instance_random_paired,ddi_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end


,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,ELM_instance_random_paired,ddi_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
0,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_0,0.887117,96.107999,77.495999,91.454999,...,None,None,None,None,A,B,None,None,None,None
1,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_1,0.871984,95.793846,73.986000,89.736111,...,None,None,None,None,A,B,None,None,None,None
2,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_2,0.760784,95.547501,57.906001,86.585239,...,None,None,None,None,A,B,None,None,None,None
3,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_3,0.413662,94.646667,21.510000,76.362500,...,None,None,None,None,A,B,None,None,None,None
4,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_4,0.359078,94.830001,19.753333,72.307001,...,None,None,None,None,A,B,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_0,0.298701,49.827778,73.161904,62.392307,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3176,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_1,0.246961,54.362000,76.415000,66.390909,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3177,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_2,0.211579,52.651538,72.910908,61.937083,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3178,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_3,0.211144,50.398334,78.798751,66.627144,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133


### 2 Finding structures without solved

In [6]:
solved_structures_DMI = [f.stem.split("_")[0] for f in (path_solved_structures / "DMI_solved_structures").iterdir() if f.suffix == ".pdb"]
solved_structures_DDI = ['_'.join(f.stem.split("_")[:3]) for f in (path_solved_structures / "DDI_solved_structures").iterdir() if f.suffix == ".pdb"]

for i, row in dataAF.iterrows():
    if (row["model_id"]) != "ranked_0":
        continue
    benchmark_set = row["benchmark_set"]
    if row["ddi_pfam_id"] is not None:
        id = row["ddi_pfam_id"] + "_" + row["PDB_id"]
        if id not in solved_structures_DDI:
            print(f"No solved structure for {id} ({benchmark_set})")
        
        if row["ddi_pfam_id_random_paired"] is not None:
            id = row["ddi_pfam_id_random_paired"] + "_" + row["PDB_id_random_paired"]
            if id not in solved_structures_DDI:
                print(f"No solved structure for {id} ({benchmark_set}, random pair)")
    else:
        if row["PDB_id"] is not None:
            id = row["PDB_id"]
            if id not in solved_structures_DMI:
                print(f"No solved structure for {id} ({benchmark_set})")
            
        if row["PDB_id_random_paired"] is not None:
            id = row["PDB_id_random_paired"]
            if id not in solved_structures_DMI:
                print(f"No solved structure for {id} ({benchmark_set}, random paired)")

No solved structure for 2CCI (random_DMI)
No solved structure for PF07724_PF00227_1G4B (random_DDI, random pair)
No solved structure for PF07724_PF00227_1G4B (random_DDI)
No solved structure for PF14978_PF00327_5OOL (random_DDI, random pair)
No solved structure for PF14978_PF00327_5OOL (random_DDI)


### 3 Fixing empty files from Joelle's export (something wrong with zips generated on a mac, don't know what)

In [8]:
different_files = []

for benchmark_set, run in {"known_DMI": "run37", "random_DMI": "run38", "mutations_DMI": "run43", "known_DDI": "run6", "random_DDI": "run5"}.items():
    set_ = "DMI" if "DMI" in benchmark_set else "DDI"
    print(benchmark_set)
    for folder in [p for p in (path_AF2_structures / set_ / benchmark_set).iterdir() if p.is_dir()]:
        print("\t", folder.name)
        if len(models_paths := [p for p in folder.iterdir() if p.suffix == ".pdb"]) != 5:
            print("\t\t", "Missing pdb files")
        for f in models_paths:
            f_luckdrive = path_af2_output /  f"AlphaFold_benchmark_{set_}" / run / f"{run}_{folder.name}" / f.name
            if not f_luckdrive.exists():
                print("\t\t", f"File on luck drive does not exist")
                continue
            
            if not filecmp.cmp(f, f_luckdrive):
                print("\t\t", f"{f.name} are different on luckdrive and local")
                different_files.append((f, f_luckdrive))

known_DMI
	 DEG_APCC_KENBOX_2_4GGD
	 DEG_COP1_1_5IGO
	 DEG_Kelch_Keap1_1_2FLU
	 DEG_Kelch_Keap1_2_3WN7
	 DEG_MDM2_SWIB_1_1YCR
	 DEG_SCF_COI1_1_3OGL
	 DEG_SCF_FBXO31_1_5VZU
	 DEG_SCF_TIR1_1_2P1Q
	 DEG_SIAH_1_2A25
	 DEG_SPOP_SBC_1_3HQM
	 DOC_AGCK_PIF_3_1ATP
	 DOC_ANK_TNKS_1_3TWU
	 DOC_CDC14_PxL_1_6G84
	 DOC_CYCLIN_RxL_1_1H25
	 DOC_GSK3_Axin_1_1O9U
	 DOC_MAPK_DCC_7_2B9J
	 DOC_MAPK_GRA24_9_5ETA
	 DOC_MAPK_HePTP_8_2GPH
	 DOC_MAPK_JIP1_4_4H3B
	 DOC_MAPK_MEF2A_6_4H3Q
	 DOC_MAPK_NFAT4_5_2XS0
	 DOC_MAPK_RevD_3_3TEI
	 DOC_MIT_MIM_1_2JQ9
	 DOC_PP1_MyPhoNE_1_1S70
	 DOC_PP1_RVXF_1_3N5U
	 DOC_PP1_SILK_1_2O8G
	 DOC_PP2B_PxIxI_1_4F0Z
	 DOC_SPAK_OSR1_1_2V3S
	 DOC_USP7_MATH_1_3MQS
	 DOC_USP7_MATH_2_1YY6
	 DOC_USP7_UBL2_3_4YOC
	 LIG_14-3-3_ChREBP_3_5F74
	 LIG_ActinCP_CPI_1_3AA0
	 LIG_ActinCP_TwfCPI_2_7DS2
	 LIG_Actin_RPEL_3_2V51
	 LIG_Actin_WH2_1_2A41
	 LIG_Actin_WH2_2_2D1K
	 LIG_ANK_PxLPxL_1_3UXG
	 LIG_AP2alpha_1_1KY7
	 LIG_AP2alpha_2_1KYU
	 LIG_APCC_ABBA_1_4BH6
	 LIG_ARL_BART_1_4ZI2
	 LIG_BH_BH3_1_2ROC

In [17]:
import shutil

for f, f_luckdrive in different_files:
    #print(f"{os.path.getsize(f):10}->{os.path.getsize(f_luckdrive):<10}", f.parent.name, f.stem)
    print(f"{os.path.getsize(f):10}", f.parent.name, f.stem)
    #shutil.copyfile(src=f_luckdrive, dst=f)

    905103 LIG_LYPXL_SIV_4_2XS1 ranked_1
    905103 LIG_LYPXL_SIV_4_2XS1 ranked_2
    905103 LIG_LYPXL_SIV_4_2XS1 ranked_3
    905103 LIG_LYPXL_SIV_4_2XS1 ranked_4
    269739 LIG_MAD2_1GO4 ranked_0
    269739 LIG_MAD2_1GO4 ranked_1
    148968 LIG_ULM_U2AF65_1_1O0P ranked_3
    148968 LIG_ULM_U2AF65_1_1O0P ranked_4
    354384 LIG_Vh1_VBS_1_1SYQ ranked_0
    354384 LIG_Vh1_VBS_1_1SYQ ranked_1
    354384 LIG_Vh1_VBS_1_1SYQ ranked_2
    354384 LIG_Vh1_VBS_1_1SYQ ranked_3
    354384 LIG_Vh1_VBS_1_1SYQ ranked_4
    384516 LIG_WD40_WDR5_VDV_1_4CY2 ranked_0
    384516 LIG_WD40_WDR5_VDV_1_4CY2 ranked_1
    384516 LIG_WD40_WDR5_VDV_1_4CY2 ranked_2
    384516 LIG_WD40_WDR5_VDV_1_4CY2 ranked_3
    384516 LIG_WD40_WDR5_VDV_1_4CY2 ranked_4
    235314 MLIG_LSD1_SNAG_1_2Y48.DLIG_EF_ALG2_ABM_1_2ZNE ranked_4
    204210 MLIG_LYPXL_L_2_2R02.DLIG_PTAP_UEV_1_1M4P ranked_0
    204210 MLIG_LYPXL_L_2_2R02.DLIG_PTAP_UEV_1_1M4P ranked_1
    204210 MLIG_LYPXL_L_2_2R02.DLIG_PTAP_UEV_1_1M4P ranked_2
    204210 MLIG

### 4 Adding domain and motif start / end from template file
While for the DDI structures selection start and end are included in the filename, for DMI structures there is absolutely no information about start/end of motif and domain. At least, the DMI structures are cut to only include the minimal domain/motif, but there still may be mutations or missing residues in experimental structures. As a SOS hack, use the id of the first and the last residue and hope the first and last residue are not missing in the solved structure. Check this later (for chain B at least) in section 5a

In [19]:
path_DMI_solved = path_solved_structures / "DMI_solved_structures"

for i, row in dataAF[dataAF["benchmark_set"].isin(['known_DMI', 'random_DMI', 'mutations_DMI'])].iterrows():
    pdb_id = str(row["PDB_id"])
    prediction_name = row["prediction_name"]
    benchmark_set = row["benchmark_set"]
    if row["PDB_id_random_paired"] is None:
        pdb_id_2 = pdb_id
    else:
        pdb_id_2 = str(row["PDB_id_random_paired"])
    
    template1_path = path_DMI_solved / (pdb_id + "_min_DMI.pdb")
    template2_path = path_DMI_solved / (pdb_id_2 + "_min_DMI.pdb")


    if template1_path.exists():
        structure_biopy = parser.get_structure("structure", file=template1_path)
        chains = [c for c in structure_biopy.get_chains()]
        if len(chains) != 2:
            print(f"Unexpected chains in {template1_path.name}")
            continue
        chainA = structure_biopy[0][chains[0].id]
        chainA_resids = [r.id[1] for r in chainA.get_residues()]
        chainA_start = np.min(chainA_resids)
        chainA_end = np.max(chainA_resids)

        dataAF.at[i, "chainA_start"] =  chainA_start
        dataAF.at[i, "chainA_end"] =  chainA_end
    else:
        print(f"Can't find template for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id} (chain A)")
    

    if template2_path.exists():
        structure_biopy = parser.get_structure("structure", file=template2_path)
        chains = [c for c in structure_biopy.get_chains()]
        if len(chains) != 2:
            print(f"Unexpected chains in {template2_path.name}")
            continue
        chainB = structure_biopy[0][chains[1].id]
        chainB_resids = [r.id[1] for r in chainB.get_residues()]
        chainB_start = np.min(chainB_resids)
        chainB_end = np.max(chainB_resids)

        dataAF.at[i, "chainB_start"] =  chainB_start
        dataAF.at[i, "chainB_end"] =  chainB_end
    else:
        print(f"Can't find template for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2} (chain B)")

dataAF

Can't find template for MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI (random_DMI) and PDB ID 2CCI (chain A)
Can't find template for MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI (random_DMI) and PDB ID 2CCI (chain A)
Can't find template for MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI (random_DMI) and PDB ID 2CCI (chain A)
Can't find template for MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI (random_DMI) and PDB ID 2CCI (chain A)
Can't find template for MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI (random_DMI) and PDB ID 2CCI (chain A)


,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,ELM_instance_random_paired,ddi_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
0,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_0,0.887117,96.107999,77.495999,91.454999,...,None,None,None,None,A,B,165,476,6,10
1,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_1,0.871984,95.793846,73.986000,89.736111,...,None,None,None,None,A,B,165,476,6,10
2,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_2,0.760784,95.547501,57.906001,86.585239,...,None,None,None,None,A,B,165,476,6,10
3,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_3,0.413662,94.646667,21.510000,76.362500,...,None,None,None,None,A,B,165,476,6,10
4,AlphaFold_benchmark,DEG_APCC_KENBOX_2_4GGD,run37,312,5,ranked_4,0.359078,94.830001,19.753333,72.307001,...,None,None,None,None,A,B,165,476,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_0,0.298701,49.827778,73.161904,62.392307,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3176,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_1,0.246961,54.362000,76.415000,66.390909,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3177,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_2,0.211579,52.651538,72.910908,61.937083,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133
3178,AlphaFold_benchmark_DDI,D1PF18773_PF00071_2X19.D2PF00009_PF01873_2D74,run6,60,113,ranked_3,0.211144,50.398334,78.798751,66.627144,...,None,PF00009_PF01873,None,None,B,B,392,451,21,133


In [21]:
dataAF[dataAF["chainA_start"].isna()]

,project_name,prediction_name,run_id,chainA_length,chainB_length,model_id,model_confidence,chainA_intf_avg_plddt,chainB_intf_avg_plddt,intf_avg_plddt,...,ELM_instance_random_paired,ddi_pfam_id_random_paired,sequence_initial,sequence_mutated,chainA_id,chainB_id,chainA_start,chainA_end,chainB_start,chainB_end
1120,AlphaFold_benchmark,MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI,run38,297,13,ranked_0,0.402063,88.842000,37.006667,69.403750,...,LIG_PAM2_1,None,None,None,A,B,None,None,4,16
1121,AlphaFold_benchmark,MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI,run38,297,13,ranked_1,0.319871,63.488750,25.925556,49.966000,...,LIG_PAM2_1,None,None,None,A,B,None,None,4,16
1122,AlphaFold_benchmark,MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI,run38,297,13,ranked_2,0.314531,61.671428,23.663750,51.186551,...,LIG_PAM2_1,None,None,None,A,B,None,None,4,16
1123,AlphaFold_benchmark,MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI,run38,297,13,ranked_3,0.281473,61.955000,26.967500,47.960000,...,LIG_PAM2_1,None,None,None,A,B,None,None,4,16
1124,AlphaFold_benchmark,MLIG_PAM2_1_1JGN.DMOD_CDK_SPxK_1_2CCI,run38,297,13,ranked_4,0.279213,58.934999,29.705001,44.320000,...,LIG_PAM2_1,None,None,None,A,B,None,None,4,16
3095,AlphaFold_benchmark_DDI,D1PF07525_PF03931_3ZKJ.D2PF07724_PF00227_1G4B,run6,48,170,ranked_0,0.260652,50.000001,86.291668,72.921054,...,None,PF07724_PF00227,None,None,None,None,None,None,None,None
3096,AlphaFold_benchmark_DDI,D1PF07525_PF03931_3ZKJ.D2PF07724_PF00227_1G4B,run6,48,170,ranked_1,0.242713,48.809333,90.618235,71.020312,...,None,PF07724_PF00227,None,None,None,None,None,None,None,None
3097,AlphaFold_benchmark_DDI,D1PF07525_PF03931_3ZKJ.D2PF07724_PF00227_1G4B,run6,48,170,ranked_2,0.240449,57.903750,87.821427,71.865333,...,None,PF07724_PF00227,None,None,None,None,None,None,None,None
3098,AlphaFold_benchmark_DDI,D1PF07525_PF03931_3ZKJ.D2PF07724_PF00227_1G4B,run6,48,170,ranked_3,0.235673,54.257500,90.602857,71.218667,...,None,PF07724_PF00227,None,None,None,None,None,None,None,None
3099,AlphaFold_benchmark_DDI,D1PF07525_PF03931_3ZKJ.D2PF07724_PF00227_1G4B,run6,48,170,ranked_4,0.233427,56.483334,77.758572,67.939231,...,None,PF07724_PF00227,None,None,None,None,None,None,None,None


In [30]:
def find_wrong_chain_metadata(check_chainA = False, check_chainB = False, remove = False):
    for i, row in dataAF[dataAF["benchmark_set"].isin(['known_DMI', 'random_DMI', 'mutations_DMI'])].iterrows():
        benchmark_set = str(row["benchmark_set"])
        _set = "DMI" if "DMI" in benchmark_set else "DDI"
        prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
        model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
        chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
        chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
        chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
        chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None

        if model_id != "ranked_0":
            continue

        structure_path = path_AF2_structures / _set / benchmark_set / prediction_name / (model_id + ".pdb")
        if not structure_path.exists():
            raise RuntimeError(f"Unexpected, the structure file for {prediction_name} is missing")
        
        try:
            structure_biopy = parser.get_structure("structure", file=structure_path)
        except Exception as ex:
            print(f"Can't open structure {prediction_name} ({benchmark_set}). Skipping")
            continue
        chains = [c for c in structure_biopy.get_chains()]
        if len(chains) != 2:
            print(f"Unexpected chains in {structure_path.name}")
            continue
        
        
        
        if check_chainA and chainA_start is not None:
            chainA = structure_biopy[0][chains[0].id]
            chainA_len = len([r for r in chainA.get_residues()])
            if (chainA_end - chainA_start + 1) != chainA_len:
                print(f"Chain A offset error for {prediction_name} ({benchmark_set}): {chainA_start}-{chainA_end} should be {chainA_len} long")
                if remove:
                    dataAF.at[i, "chainA_start"] = None
                    dataAF.at[i, "chainA_end"] = None

        if check_chainB and chainB_start is not None:
            chainB = structure_biopy[0][chains[1].id]
            chainB_len = len([r for r in chainB.get_residues()])
            if (chainB_end - chainB_start + 1) != chainB_len:
                print(f"Chain B offset error for {prediction_name} ({benchmark_set}): {chainB_start}-{chainB_end} should be {chainB_len} long")
                if remove:
                    dataAF.at[i, "chainB_start"] = None
                    dataAF.at[i, "chainB_end"] = None


In [23]:
find_wrong_chain_metadata(check_chainB=True)

Chain B offset error for DEG_SCF_COI1_1_3OGL (known_DMI): 203-217 should be 18 long
Chain B offset error for DOC_MAPK_GRA24_9_5ETA (known_DMI): -2-13 should be 15 long
Chain B offset error for LIG_LRP6_Inhibitor_1_3SOQ (known_DMI): 38-43 should be 7 long
Chain B offset error for LIG_NRP_CendR_1_2ORZ (known_DMI): 2-4 should be 4 long
Chain B offset error for MDEG_SCF_COI1_1_3OGL.DLIG_CaM_IQ_9_2IX7 (random_DMI): 203-217 should be 18 long
Chain B offset error for MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5MLW (random_DMI): -2-13 should be 15 long
Chain B offset error for MLIG_LRP6_Inhibitor_1_3SOQ.DTRG_NES_CRM1_1_3GB8 (random_DMI): 38-43 should be 7 long
Chain B offset error for MLIG_MYND_2_2ODD.DMOD_SUMO_for_1_1KPS (random_DMI): 1105-1109 should be 41 long
Chain B offset error for MLIG_NRP_CendR_1_2ORZ.DLIG_KLC1_Yacidic_2_6FUZ (random_DMI): 2-4 should be 4 long
Chain B offset error for MLIG_Pex14_1_2W84.DLIG_PROFILIN_1_2V8C (random_DMI): 103-107 should be 4 long
Chain B offset error for DE

From here on it is manual fixing
* DEG_SCF_COI1_1_3OGL (known_minimal): The proper end should be 220
* DOC_MAPK_GRA24_9_5ETA (known_minimal): The proper start should be -1
* LIG_LRP6_Inhibitor_1_3SOQ (known_minimal): The proper start should be 37
* LIG_NRP_CendR_1_2ORZ (known_minimal): The proper start should be 1
* MLIG_MYND_2_2ODD.DMOD_SUMO_for_1_1KPS (random_minimal): Wrong label! 2ODD is NOT contained in the AF3 output - Discard chain information for chain B
* MLIG_Pex14_1_2W84.DLIG_PROFILIN_1_2V8C (random_minimal): The proper end should be 106. But ONLY for random_minimal
For mutations and random minimal the behaviour is the same

In [26]:
for i, row in dataAF[dataAF["benchmark_set"].isin(['known_DMI', 'random_DMI', 'mutations_DMI'])].iterrows():
    pdb_id = str(row["PDB_id"])
    benchmark_set = str(row["benchmark_set"])
    if benchmark_set == "random_DMI":
        pdb_id = str(row["PDB_id_random_paired"])

    chainB_start, chainB_end = row["chainB_start"], row["chainB_end"]
    match pdb_id:
        case "3OGL":
            chainB_end = 220
        case "5ETA":
            chainB_start = -1
        case "3SOQ":
            chainB_start = 37
        case "2ORZ":
            chainB_start = 1
        case "2ODD":
            if benchmark_set != "random_DMI": continue
            chainB_start = None
            chainB_end = None
        case "2W84":
            if benchmark_set != "random_DMI": continue
            chainB_end = 106
        case _:
            continue
    if row["model_id"] == "ranked_0":
        print(row["prediction_name"])
    
    dataAF.at[i, "chainB_start"] = chainB_start
    dataAF.at[i, "chainB_end"] = chainB_end

DEG_SCF_COI1_1_3OGL
DOC_MAPK_GRA24_9_5ETA
LIG_LRP6_Inhibitor_1_3SOQ
LIG_NRP_CendR_1_2ORZ
MDEG_SCF_COI1_1_3OGL.DLIG_CaM_IQ_9_2IX7
MDOC_MAPK_GRA24_9_5ETA.DLIG_PCNA_APIM_2_5MLW
MLIG_LRP6_Inhibitor_1_3SOQ.DTRG_NES_CRM1_1_3GB8
MLIG_MYND_2_2ODD.DMOD_SUMO_for_1_1KPS
MLIG_NRP_CendR_1_2ORZ.DLIG_KLC1_Yacidic_2_6FUZ
MLIG_Pex14_1_2W84.DLIG_PROFILIN_1_2V8C
DEG_SCF_COI1_1_3OGL_IARRASLHRFLEKRKDRV.IARRASLHRFDEKRKDGV
DEG_SCF_COI1_1_3OGL_IARRASLHRFLEKRKDRV.IARRASLHRFDEKRKDRV
DOC_MAPK_GRA24_9_5ETA_LLERRGVSELPPLYI.LLERGGVSELPPDYI
DOC_MAPK_GRA24_9_5ETA_LLERRGVSELPPLYI.LLERGGVSELPPLYI
LIG_LRP6_Inhibitor_1_3SOQ_LNSNAIK.LNSNADG
LIG_LRP6_Inhibitor_1_3SOQ_LNSNAIK.LNSNAIG
LIG_NRP_CendR_1_2ORZ_TKPR.TGPG
LIG_NRP_CendR_1_2ORZ_TKPR.TKPG


In [27]:
find_wrong_chain_metadata(check_chainB=True)

In [28]:
find_wrong_chain_metadata(check_chainA=True)

Chain A offset error for DEG_SCF_TIR1_1_2P1Q (known_DMI): 45-576 should be 534 long
Chain A offset error for DEG_SIAH_1_2A25 (known_DMI): 125-282 should be 193 long
Chain A offset error for DEG_SPOP_SBC_1_3HQM (known_DMI): 29-164 should be 137 long
Chain A offset error for LIG_deltaCOP1_diTrp_1_5FJX (known_DMI): 286-546 should be 259 long
Chain A offset error for LIG_KLC1_WD_1_3ZFW (known_DMI): 236-479 should be 245 long
Chain A offset error for LIG_MAD2_1GO4 (known_DMI): 10-203 should be 196 long
Chain A offset error for LIG_MYND_2_2ODD (known_DMI): 661-701 should be 51 long
Chain A offset error for LIG_RPA_C_Vert_1DPU (known_DMI): 202-264 should be 93 long
Chain A offset error for LIG_SUFU_1_4KMD (known_DMI): 62-475 should be 354 long
Chain A offset error for LIG_SUMO_SIM_par_1_2ASQ (known_DMI): 21-93 should be 74 long
Chain A offset error for LIG_WD40_WDR5_WIN_2_4CY3 (known_DMI): 59-361 should be 312 long
Chain A offset error for LIG_WW_1_1EG4 (known_DMI): 47-306 should be 261 long


On chain A are various errors. But this ok, as they are not needed. But remove them in case someone is reusing the data

In [32]:
find_wrong_chain_metadata(check_chainA=True, remove=True)

### 2 Pymol calculations

In [ ]:
# Calculating the RMSD related values using pymol

dataAF["RMSD_domain"] = None
dataAF["align_score_domain"] = None
dataAF["num_align_atoms_domain"] = None
dataAF["num_align_resi_domain"] = None
dataAF["RMSD_backbone_peptide"] = None
dataAF["RMSD_all_atom_peptide"] = None
dataAF["RMSD_DDI_max"] = None
dataAF["RMSD_all_atom"] = None

for i,row in dataAF.iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if "DDI" in benchmark_set else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 = str(row["PDB_id_random_paired"]) if row.notnull()["PDB_id_random_paired"] else None
    ddi_pfam_id = str(row["ddi_pfam_id"]) if row.notnull()["ddi_pfam_id"] else None
    ddi_pfam_id_2 = str(row["ddi_pfam_id_random_paired"]) if row.notnull()["ddi_pfam_id_random_paired"] else None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None

    if model_id == "ranked_0":
        print(prediction_name, benchmark_set)    

    structure_path = path_AF2_structures / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    if not structure_path.exists():
        raise RuntimeError(f"For {prediction_name}/{model_id} the structure file is missing. Skip this structure")
    
    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["ddi_pfam_id"] == ddi_pfam_id, dataSolved["ddi_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\tCan't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip RMSD calculation")
        continue
    elif len(template_row) >= 2:
        print(f"\tMultiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip RMSD calculation")
        continue

    template_path = path_solved_structures / str(template_row["path"].item())
    if not template_path.exists():
        if model_id == "ranked_0":
            print(f"\tSkipping {prediction_name} as there is no template file")
        continue

    template2_path = None
    if pdb_id_2 is not None:
        template2_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id_2, np.logical_or(dataSolved["ddi_pfam_id"] == ddi_pfam_id_2, dataSolved["ddi_pfam_id"].isna())))]
        if len(template2_row) == 0:
            print(f"\tCan't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}. Skip RMSD calculation")
            continue
        elif len(template2_row) >= 2:
            print(f"\tMultiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id_2}. Skip RMSD calculation")
            continue

        template2_path = path_solved_structures / str(template2_row["path"].item())
        if not template2_path.exists():
            if model_id == "ranked_0":
                print(f"\tSkipping {prediction_name} as there is no template")
            continue

    #pymol.cmd.reinitialize() # Not needed usually, but slows performance significantly down
    for o in pymol.cmd.get_object_list():
        pymol.cmd.delete("all")
    pymol.cmd.sort()

    # First loading the structures. Use two temporary objects to allow renaming the chains even if the chains have the same name or have switched IDs
    pymol.cmd.load(structure_path, "af")
    if template2_path is not None:
        # Updating the object is possible, but turned out to be unstable
        pymol.cmd.load(template_path, "solvedA")
        pymol.cmd.load(template2_path, "solvedB")
        pymol.cmd.create("solved1", f"solvedA and chain {chainA_id}")
        pymol.cmd.create("solved2", f"solvedB and chain {chainB_id}")
        pymol.cmd.delete("solvedA")
        pymol.cmd.delete("solvedB")
    else:
        pymol.cmd.load(template_path, "solvedraw")
        pymol.cmd.create("solved1", f"solvedraw and chain {chainA_id}")
        pymol.cmd.sort()
        pymol.cmd.create("solved2", f"solvedraw and chain {chainB_id}")
        pymol.cmd.delete("solvedraw")
    pymol.cmd.sort()
    # Now rename the chains and create merged object
    pymol.cmd.alter(f"solved1 and chain {chainA_id}", "chain = 'A'")
    pymol.cmd.sort()
    pymol.cmd.alter(f"solved2 and chain {chainB_id}", "chain = 'B'")
    pymol.cmd.sort()
    pymol.cmd.create("solved", f"solved1 or solved2")
    pymol.cmd.delete("solved1")
    pymol.cmd.delete("solved2")
    pymol.cmd.sort()

    # Remove hydrogens and hetatm
    pymol.cmd.remove(selection="elem 'H' or hetatm")
    pymol.cmd.sort()

    # Remove alternate location identifiers
    pymol.cmd.remove("not alt ''+A") # Using +A syntax to only effect the atoms with an alternate location identifier set
    pymol.cmd.sort()
    pymol.cmd.alter("all", "alt=''")
    pymol.cmd.sort()

    # Slice the chains to the known start/end residues. For chain B and AF a reindexing is performed as the rms_cur cmd of pymol requires same residue numbers for alignment
    if chainA_start is not None and chainB_start is not None:
        pymol.cmd.create("solved", f"solved and ((chain A and resi {chainA_start}-{chainA_end}) or (chain B and resi {chainB_start}-{chainB_end}))", source_state=0, target_state=0)
        pymol.cmd.sort()
        offset = chainB_start - 1
        pymol.cmd.alter("af and chain B", f"resi = (int(resi) + {offset})")
        pymol.cmd.sort()
    else:
        print(f"\tCan't find information about the chain start/end in the template. This may lead to wrong RMSD peptide values, so skip")
        continue

    pymol.cmd.sort()

    #For debugging
    #space = {'solved_resi': [], "af_resi": []}
    #pymol.cmd.iterate("solved and chain B", "solved_resi.append(int(resi))", space=space)
    #pymol.cmd.iterate("af and chain B", "af_resi.append(int(resi))", space=space)

    #    0: RMSD after refinement
    #    1: Number of aligned atoms after refinement
    #    2: Number of refinement cycles
    #    3: RMSD before refinement
    #    4: Number of aligned atoms before refinement
    #    5: Raw alignment score
    #    6: Number of residues aligned
    # Cycles = 0 to prevent rejection of outliers
    align_output_1 = pymol.cmd.align(mobile="af and chain A", target="solved and chain A", object="algn_domain", cycles=0)
    pymol.cmd.sort()
    RMSD_domain = align_output_1[0]
    num_align_atoms_domain = align_output_1[1]
    align_score_domain = align_output_1[5]
    num_align_resi_domain = align_output_1[6]

    if _set == "DDI":
        align_output_2 = pymol.cmd.align(mobile="af and chain B", target="solved and chain B", object="algn_domain2", cycles=0, )
        RMSD_domain_2 = align_output_2[0]
        dataAF.at[i, "RMSD_DDI_max"] = np.max([RMSD_domain, RMSD_domain_2])
    elif _set == "DMI":
        dataAF.at[i, "RMSD_domain"] =  RMSD_domain
        dataAF.at[i, "align_score_domain"] =  align_score_domain
        dataAF.at[i, "num_align_atoms_domain"] =  num_align_atoms_domain
        dataAF.at[i, "num_align_resi_domain"] =  num_align_resi_domain

    align_output = pymol.cmd.align(mobile="af", target="solved", object="algn_domain3", cycles=0, )
    RMSD_all_atoms = align_output[0]
    dataAF.at[i, "RMSD_all_atom"] =  RMSD_all_atoms

    if benchmark_set in ["known_minimal", "mutations"]:
        RMSD_backbone_peptide = pymol.cmd.rms_cur(mobile="af and chain B and bb.", target="solved and chain B and bb.", object="peptide_super_bb")
        RMSD_all_atom_peptide = pymol.cmd.rms_cur(mobile="af and chain B", target="solved and chain B", object="peptide_super_all_atoms")
        dataAF.at[i, "RMSD_backbone_peptide"] =  RMSD_backbone_peptide
        dataAF.at[i, "RMSD_all_atom_peptide"] =  RMSD_all_atom_peptide

NameError: name 'dataAF' is not defined

In [ ]:
display(dataAF)

,model_preset,benchmark_set,prediction_name,model_id,num_mutations,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,chains_flipped,model_path,RMSD_domain,align_score_domain,num_align_atoms_domain,num_align_resi_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,RMSD_DDI_max,RMSD_all_atom
0,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_0,NaN,0.28,189,113,0.04,0.0,...,True,AlphaFold_benchmark_DDI\known_ddi\suspicious_c...,None,None,None,None,None,None,5.313323,15.168005
1,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_1,NaN,0.25,189,113,0.04,0.0,...,True,AlphaFold_benchmark_DDI\known_ddi\suspicious_c...,None,None,None,None,None,None,4.992111,15.852812
2,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_2,NaN,0.22,189,113,0.04,0.0,...,True,AlphaFold_benchmark_DDI\known_ddi\suspicious_c...,None,None,None,None,None,None,4.29104,15.517168
3,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_3,NaN,0.19,189,113,0.04,0.0,...,True,AlphaFold_benchmark_DDI\known_ddi\suspicious_c...,None,None,None,None,None,None,4.732747,16.243053
4,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_4,NaN,0.17,189,113,0.04,0.0,...,True,AlphaFold_benchmark_DDI\known_ddi\suspicious_c...,None,None,None,None,None,None,4.57719,16.653427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,NaN,0.93,312,4,0.02,0.0,...,True,AlphaFold_benchmark_DMI\random_minimal\gloomy_...,0.684738,1624.0,2356,303,None,None,None,7.710459
3176,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,NaN,0.92,312,4,0.01,0.0,...,True,AlphaFold_benchmark_DMI\random_minimal\gloomy_...,0.684467,1624.0,2356,303,None,None,None,7.707559
3177,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,NaN,0.92,312,4,0.02,0.0,...,True,AlphaFold_benchmark_DMI\random_minimal\gloomy_...,1.018179,1624.0,2356,303,None,None,None,7.750147
3178,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,NaN,0.91,312,4,0.01,0.0,...,True,AlphaFold_benchmark_DMI\random_minimal\gloomy_...,0.658395,1624.0,2356,303,None,None,None,7.704717


### 3 DockQ


In [ ]:
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces

dataAF["DockQ"] = np.nan
dataAF["iRMSD"] = np.nan
dataAF["LRMSD"] = np.nan
dataAF["Fnonnat"] = np.nan
for i, row in dataAF[dataAF["benchmark_set"].isin(["known_minimal", "known_ddi"])].iterrows():
    benchmark_set = str(row["benchmark_set"])
    _set = "DDI" if benchmark_set in ["known_ddi", "random_ddi"] else "DMI"
    pdb_id = str(row["PDB_id"]) if row.notnull()["PDB_id"] else None
    pdb_id_2 = str(row["PDB_id_random_paired"]) if row.notnull()["PDB_id_random_paired"] else None
    ddi_pfam_id = str(row["ddi_pfam_id"]) if row.notnull()["ddi_pfam_id"] else None
    ddi_pfam_id_2 = str(row["ddi_pfam_id_random_paired"]) if row.notnull()["ddi_pfam_id_random_paired"] else None
    prediction_name = str(row["prediction_name"]) if row.notnull()["prediction_name"] else None
    model_id = str(row["model_id"]) if row.notnull()["model_id"] else None
    chainA_id = str(row["chainA_id"]) if row.notnull()["chainA_id"] else None
    chainB_id = str(row["chainB_id"]) if row.notnull()["chainB_id"] else None
    chainA_start = int(row["chainA_start"]) if row.notnull()["chainA_start"] else None
    chainB_start = int(row["chainB_start"]) if row.notnull()["chainB_start"] else None
    chainA_end = int(row["chainA_end"]) if row.notnull()["chainA_end"] else None
    chainB_end = int(row["chainB_end"]) if row.notnull()["chainB_end"] else None

    if model_id == "ranked_0":
        print(f"{prediction_name} ({benchmark_set})")

    template_row = dataSolved.loc[np.logical_and(dataSolved["set"] == _set, np.logical_and(dataSolved["PDB_id"] == pdb_id, np.logical_or(dataSolved["ddi_pfam_id"] == ddi_pfam_id, dataSolved["ddi_pfam_id"].isna())))]
    if len(template_row) == 0:
        print(f"\tCan't find template structure for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip")
        continue
    elif len(template_row) >= 2:
        print(f"\tMultiple template structures found for {prediction_name} ({benchmark_set}) and PDB ID {pdb_id}. Skip")
        continue

    structure_path = path_AF3_structures / _set / benchmark_set / prediction_name / (model_id + ".pdb")
    template_path = path_solved_structures / str(template_row["path"].item())
    dockq_structure_af = load_PDB(str(structure_path))
    dockq_structure_solved = load_PDB(str(template_path))

    chain_map = {chainA_id: "A", chainB_id:"B"}
    chain_key = chainA_id + chainB_id

    result = run_on_all_native_interfaces(dockq_structure_af, dockq_structure_solved, chain_map=chain_map)[0]
    dataAF.at[i, "DockQ"] = result[chain_key]["DockQ"]
    dataAF.at[i, "iRMSD"] = result[chain_key]["iRMSD"]
    dataAF.at[i, "LRMSD"] = result[chain_key]["LRMSD"]
    dataAF.at[i, "Fnonnat"] = np.float64(result[chain_key]["fnonnat"])

display(dataAF)


PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133 (known_ddi)
PF00026_PF06394_1F34_A_resi13_resi326.B_resi62_resi120 (known_ddi)
PF00059_PF00041_1TDQ_B_resi10_resi125.A_resi85_resi186 (known_ddi)
PF00089_PF00095_1FLE_E_resi16_resi243.I_resi12_resi56 (known_ddi)
PF00137_PF07850_6VQG_i_resi7_resi86.p_resi292_resi343 (known_ddi)
PF00244_PF01161_3AXY_J_resi4_resi233.H_resi19_resi169 (known_ddi)
PF00289_PF02436_5VYW_A_resi1_resi120.D_resi810_resi1030 (known_ddi)
PF00454_PF00017_2Y3A_A_resi794_resi1010.B_resi616_resi690 (known_ddi)
PF00514_PF00104_3TX7_A_resi148_resi661.B_resi316_resi533 (known_ddi)
PF00675_PF02271_1PP9_B_resi35_resi180.S_resi12_resi105 (known_ddi)
PF00787_PF03643_5F0L_B_resi8_resi283.C_resi58_resi147 (known_ddi)
PF00858_PF00087_7CFT_A_resi48_resi461.D_resi1_resi56 (known_ddi)
PF00890_PF13085_1L0V_M_resi1_resi406.N_resi2_resi121 (known_ddi)
PF01298_PF00405_3VE1_A_resi174_resi345.B_resi342_resi664 (known_ddi)
PF02351_PF17812_6Q2N_D_resi243_resi337.F_resi265_resi379 (know

,model_preset,benchmark_set,prediction_name,model_id,num_mutations,ranking_score,chainA_length,chainB_length,fraction_disordered,has_clash,...,num_align_atoms_domain,num_align_resi_domain,RMSD_backbone_peptide,RMSD_all_atom_peptide,RMSD_DDI_max,RMSD_all_atom,DockQ,iRMSD,LRMSD,Fnonnat
0,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_0,NaN,0.28,189,113,0.04,0.0,...,None,None,None,None,5.313323,15.168005,0.050360,13.928462,24.152736,0.884615
1,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_1,NaN,0.25,189,113,0.04,0.0,...,None,None,None,None,4.992111,15.852812,0.047522,15.915324,24.902460,0.884615
2,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_2,NaN,0.22,189,113,0.04,0.0,...,None,None,None,None,4.29104,15.517168,0.048263,15.056429,24.743910,0.892857
3,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_3,NaN,0.19,189,113,0.04,0.0,...,None,None,None,None,4.732747,16.243053,0.049288,14.866893,25.704516,0.894737
4,alphafold3,known_ddi,PF00009_PF01873_2D74_A_resi12_resi200.B_resi21...,ranked_4,NaN,0.17,189,113,0.04,0.0,...,None,None,None,None,4.57719,16.653427,0.023410,17.734430,32.745436,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3175,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_0,NaN,0.93,312,4,0.02,0.0,...,2356,303,None,None,None,7.710459,NaN,NaN,NaN,NaN
3176,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_1,NaN,0.92,312,4,0.01,0.0,...,2356,303,None,None,None,7.707559,NaN,NaN,NaN,NaN
3177,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_2,NaN,0.92,312,4,0.02,0.0,...,2356,303,None,None,None,7.750147,NaN,NaN,NaN,NaN
3178,alphafold3,random_minimal,MTRG_PTS1_2C0L.DLIG_WD40_WDR5_WIN_2_4CY3,ranked_3,NaN,0.91,312,4,0.01,0.0,...,2356,303,None,None,None,7.704717,NaN,NaN,NaN,NaN


### 4 IPSAE metric

In [ ]:
def calc_ipsae_metric(row: pd.Series):
    path_cif = luck_drive_folder / "AlphaFold3" / Path(row["model_path"])
    path_confidences = path_cif.parent / "confidences.json"

    with tempfile.TemporaryDirectory() as tmpdir:

        shutil.copy(path_cif, tmp_path_cif := (Path(tmpdir) / "model.cif"))
        shutil.copy(path_confidences, tmp_path_confidences := (Path(tmpdir) / "confidences.json"))
        subprocess.run(["python", path_ipsae_script, tmp_path_confidences, tmp_path_cif, "10", "10"], env=os.environ.copy())

        path_output = Path(tmpdir) / "model_10_10.txt"

        df_ipsae = pd.read_csv(path_output, header=0, skiprows=[0], sep=" ", skipinitialspace=True)

        print(tmpdir, flush=True)
        if row["prediction_name"] == "PF00059_PF00041_1TDQ_B_resi10_resi125.A_resi85_resi186":
            input("Wait...")
    return df_ipsae

dataAF["ipSAE"] = np.nan
for i, row in dataAF.iterrows():
    if row["model_id"] == "ranked_0":
        print(row["prediction_name"], f"({round(100*i/len(dataAF))} %)")
    df_ipsae = calc_ipsae_metric(row)
    dataAF.at[i, "ipSAE"] = np.float64(df_ipsae["ipSAE"][2])
display(dataAF)

PF00009_PF01873_2D74_A_resi12_resi200.B_resi21_resi133 (0 %)
C:\Users\abril\AppData\Local\Temp\tmp1d2h68ms
C:\Users\abril\AppData\Local\Temp\tmpt31_h_z8
C:\Users\abril\AppData\Local\Temp\tmp8shx7bmb
C:\Users\abril\AppData\Local\Temp\tmp_e0z1idu
C:\Users\abril\AppData\Local\Temp\tmpsjibxc_q
PF00026_PF06394_1F34_A_resi13_resi326.B_resi62_resi120 (0 %)
C:\Users\abril\AppData\Local\Temp\tmp4hti5jy5
C:\Users\abril\AppData\Local\Temp\tmp1n54ut_z
C:\Users\abril\AppData\Local\Temp\tmpyi54elmz
C:\Users\abril\AppData\Local\Temp\tmpzfg3fckc
C:\Users\abril\AppData\Local\Temp\tmpo5od6r74
PF00059_PF00041_1TDQ_B_resi10_resi125.A_resi85_resi186 (0 %)
C:\Users\abril\AppData\Local\Temp\tmp44wq7kcl


KeyboardInterrupt: Interrupted by user

### Saving

In [33]:
# Save dataframe
dataAF.to_csv(path_AF2_structures / "AF2_metrics.tsv", sep="\t", index=None)